In [ ]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 112kB 7.4MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import tensorflowjs as tfjs

from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [ ]:
dataset=pd.read_csv('https://raw.githubusercontent.com/taposh/talks/master/test.csv')
print(dataset.shape)
dataset.head()

(80, 25)


,ID,age,blood_pressure,specific_gravity,albumin,sugar,redbloodcells,pus_cell,pus_cell_clump,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_vol,wbcc,rbcc,htn,dm,cad,appet,pe,ane
0,1,48,80,1.02,1,0,?,normal,notpresent,notpresent,121,36,1.2,?,?,15.4,44,7800,5.2,yes,yes,no,good,no,no
1,5,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,26,1.4,?,?,11.6,35,7300,4.6,no,no,no,good,no,no
2,6,60,90,1.015,3,0,?,?,notpresent,notpresent,74,25,1.1,142,3.2,12.2,39,7800,4.4,yes,yes,no,good,yes,no
3,8,24,?,1.015,2,4,normal,abnormal,notpresent,notpresent,410,31,1.1,?,?,12.4,44,6900,5,no,yes,no,good,yes,no
4,10,53,90,1.02,2,0,abnormal,abnormal,present,notpresent,70,107,7.2,114,3.7,9.5,29,12100,3.7,yes,yes,no,poor,no,yes


In [ ]:
for col in dataset.columns:
  dataset=dataset[dataset[col]!='?']

In [ ]:
print(dataset.shape)
dataset.head()

(31, 25)


,ID,age,blood_pressure,specific_gravity,albumin,sugar,redbloodcells,pus_cell,pus_cell_clump,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_vol,wbcc,rbcc,htn,dm,cad,appet,pe,ane
4,10,53,90,1.02,2,0,abnormal,abnormal,present,notpresent,70,107,7.2,114,3.7,9.5,29,12100,3.7,yes,yes,no,poor,no,yes
6,21,61,80,1.015,2,0,abnormal,abnormal,notpresent,notpresent,173,148,3.9,135,5.2,7.7,24,9200,3.2,yes,yes,yes,poor,yes,yes
25,134,70,100,1.015,4,0,normal,normal,notpresent,notpresent,118,125,5.3,136,4.9,12,37,8400,8,yes,no,no,good,no,no
32,177,21,90,1.01,4,0,normal,abnormal,present,present,107,40,1.7,125,3.5,8.3,23,12400,3.9,no,no,no,good,no,yes
39,213,40,70,1.015,3,4,normal,normal,notpresent,notpresent,253,150,11.9,132,5.6,10.9,31,8800,3.4,yes,yes,no,poor,yes,no


In [ ]:
dataset.dtypes

ID                        int64
age                      object
blood_pressure           object
specific_gravity         object
albumin                  object
sugar                    object
redbloodcells            object
pus_cell                 object
pus_cell_clump           object
bacteria                 object
blood_glucose_random     object
blood_urea               object
serum_creatinine        float64
sodium                   object
potassium                object
hemoglobin               object
packed_cell_vol          object
wbcc                     object
rbcc                     object
htn                      object
dm                       object
cad                      object
appet                    object
pe                       object
ane                      object
dtype: object

In [ ]:
numerical_columns=["age","blood_pressure","specific_gravity",
                   "albumin","sugar","blood_glucose_random",
                   "blood_urea","serum_creatinine",
                   "sodium","potassium","hemoglobin",
                   "packed_cell_vol","wbcc","rbcc"]

In [ ]:
# for col in dataset.columns:
#   if col in numerical_columns:
#     dataset[col].astype(str).astype(float)

for col in numerical_columns:
  dataset[col]=dataset[col].astype(float)

In [ ]:
dataset.dtypes

ID                        int64
age                     float64
blood_pressure          float64
specific_gravity        float64
albumin                 float64
sugar                   float64
redbloodcells            object
pus_cell                 object
pus_cell_clump           object
bacteria                 object
blood_glucose_random    float64
blood_urea              float64
serum_creatinine        float64
sodium                  float64
potassium               float64
hemoglobin              float64
packed_cell_vol         float64
wbcc                    float64
rbcc                    float64
htn                      object
dm                       object
cad                      object
appet                    object
pe                       object
ane                      object
dtype: object

In [ ]:
dataset.head()

,ID,age,blood_pressure,specific_gravity,albumin,sugar,redbloodcells,pus_cell,pus_cell_clump,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_vol,wbcc,rbcc,htn,dm,cad,appet,pe,ane
4,10,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,107.0,7.2,114.0,3.7,9.5,29.0,12100.0,3.7,yes,yes,no,poor,no,yes
6,21,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,148.0,3.9,135.0,5.2,7.7,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes
25,134,70.0,100.0,1.015,4.0,0.0,normal,normal,notpresent,notpresent,118.0,125.0,5.3,136.0,4.9,12.0,37.0,8400.0,8.0,yes,no,no,good,no,no
32,177,21.0,90.0,1.010,4.0,0.0,normal,abnormal,present,present,107.0,40.0,1.7,125.0,3.5,8.3,23.0,12400.0,3.9,no,no,no,good,no,yes
39,213,40.0,70.0,1.015,3.0,4.0,normal,normal,notpresent,notpresent,253.0,150.0,11.9,132.0,5.6,10.9,31.0,8800.0,3.4,yes,yes,no,poor,yes,no


In [ ]:
dataset.to_csv('ckd_test_cleanded.csv', index=False)

###### Transform the non-numeric data in the columns

In [ ]:
for col in dataset.columns:
  if dataset[col].dtype==np.number:
    continue
  dataset[col]=LabelEncoder().fit_transform(dataset[col])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  


In [ ]:
dataset

,ID,age,blood_pressure,specific_gravity,albumin,sugar,redbloodcells,pus_cell,pus_cell_clump,bacteria,blood_glucose_random,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_vol,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
1,103,28.0,60.0,1.025,0.0,0.0,1,1,0,0,79.0,50.0,0.5,145.0,5.0,17.6,51.0,6500.0,5.0,0,0,0,0,0,0,1
2,82,32.0,70.0,1.025,0.0,0.0,1,1,0,0,100.0,29.0,1.1,142.0,4.5,14.3,43.0,6700.0,5.9,0,0,0,0,0,0,1
4,72,57.0,60.0,1.020,0.0,0.0,1,1,0,0,105.0,49.0,1.2,150.0,4.7,15.7,44.0,10400.0,6.2,0,0,0,0,0,0,1
10,89,39.0,60.0,1.020,0.0,0.0,1,1,0,0,86.0,37.0,0.6,150.0,5.0,13.6,51.0,5800.0,4.5,0,0,0,0,0,0,1
15,84,64.0,60.0,1.020,0.0,0.0,1,1,0,0,106.0,27.0,0.7,150.0,3.3,14.4,42.0,8100.0,4.7,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,87,38.0,80.0,1.020,0.0,0.0,1,1,0,0,99.0,19.0,0.5,147.0,3.5,13.6,44.0,7300.0,6.4,0,0,0,0,0,0,1
315,4,69.0,70.0,1.010,3.0,4.0,1,0,0,0,264.0,87.0,2.7,130.0,4.0,12.5,37.0,9600.0,4.1,1,1,1,0,1,0,0
316,42,42.0,80.0,1.020,0.0,0.0,1,1,0,0,98.0,20.0,0.5,140.0,3.5,13.9,44.0,8400.0,5.5,0,0,0,0,0,0,1
318,90,32.0,60.0,1.025,0.0,0.0,1,1,0,0,102.0,17.0,0.4,147.0,4.7,14.6,41.0,6800.0,5.1,0,0,0,0,0,0,1


In [ ]:
#SPLITTING INTO DEPENDENT AND INDEPENDENT SET
X=dataset.drop(["ID","class"], axis=1)
Y=dataset['class']

In [ ]:
#SCALING THE DATA
X_scaler=MinMaxScaler()
X_scaler.fit(X)
column_names=X.columns
X[column_names]=X_scaler.transform(X)

In [ ]:
#BUILD THE MODEL
model=Sequential()
model.add(Dense(256, input_dim=len(X.columns),activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
#COMPILE THE MODEL
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history=model.fit(X,Y,epochs=200, validation_split=0.1,batch_size=X.shape[0])

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.6857 - accuracy: 0.7982 - val_loss: 0.6579 - val_accuracy: 0.7692
Epoch 2/200
1/1 [==============================] - 0s 46ms/step - loss: 0.6480 - accuracy: 0.8421 - val_loss: 0.6174 - val_accuracy: 0.7692
Epoch 3/200
1/1 [==============================] - 0s 48ms/step - loss: 0.6125 - accuracy: 0.8596 - val_loss: 0.5790 - val_accuracy: 0.7692
Epoch 4/200
1/1 [==============================] - 0s 43ms/step - loss: 0.5794 - accuracy: 0.8947 - val_loss: 0.5430 - val_accuracy: 0.8462
Epoch 5/200
1/1 [==============================] - 0s 43ms/step - loss: 0.5484 - accuracy: 0.9123 - val_loss: 0.5096 - val_accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 48ms/step - loss: 0.5197 - accuracy: 0.9123 - val_loss: 0.4784 - val_accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 48ms/step - loss: 0.4930 - accuracy: 0.9035 - val_loss: 0.4493 - val_accuracy: 1.0000
Epoch 8/200
1/1 [=====

In [ ]:
#Set Model Path
from datetime import date
import os
today = date.today()
# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%Y-%H-%M-%S")
name = "ckd_" + d4
module_path = os.path.join('sample_data/' + name)
if not os.path.exists(module_path):
    os.mkdir(module_path, mode = 0o777, dir_fd = None)
print(module_path)

output_path = os.path.join(module_path+'/'+'out')
if not os.path.exists(output_path):
    os.mkdir(output_path, mode = 0o777, dir_fd = None)

print(output_path)

sample_data/ckd_Mar-30-2021-00-00-00
sample_data/ckd_Mar-30-2021-00-00-00/out


In [ ]:
#SAVING THE MODEL
model.save(os.path.join(module_path, "model.h5"))

In [ ]:
#loading the model
model2=keras.models.load_model(module_path +"/model.h5")


In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6400      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 6,657
Trainable params: 6,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#CONTERTING INTO JSON OBJECT:
#this will create a model.json file in the output directory 
#this can be loaded in the js file using tf.loadLayersModel('url') function
# tfjs.converters.save_keras_model(model,output_path)

In [ ]:
!tensorflowjs_converter \
    --input_format keras \
    --skip_op_check --saved_model_tags=serve \
    --strip_debug_ops=True --weight_shard_size_bytes=4194304\
    "/content/sample_data/ckd_Mar-30-2021-00-00-00/model.h5"\
    "/content/sample_data/ckd_Mar-30-2021-00-00-00/out"

2021-03-30 01:32:31.379006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
